In [22]:
# -*- coding: utf-8 -*-
%matplotlib inline

import pandas as pd
import numpy as np
import math
import matplotlib as mplstyle
import matplotlib.pyplot as plt
import locale
import arrow
from locale import atof
from dateutil import parser
from datetime import datetime

### Functions

In [23]:
# Create custom column: checks if one exists, true else take other
# check for ISO_A3
# if empty check if adm0_dif is 1
    # if 1 take gu_a3 
    # if 0 take adm0_a3
def mergeCol(isoa3,adm0dif,level,adm0a3,adm0a3is,gua3):
    if (isoa3 != '-99'):
        return isoa3
    else:
        if (adm0dif == 1 and level == 3):
            if (adm0a3is != '-99'):
                return adm0a3is
            else:
                return adm0a3
        elif (adm0dif == 0 and level == 3):
            return adm0a3
        elif (adm0dif == 0 and level == 2):
            try:
                if (adm0a3is != '-99'):
                    return adm0a3is
            except:
                return gua3

### Imports

In [24]:
df_graphicExample = pd.read_csv('../data/imports/graphic-copy-sample.csv')
df_completeCountryList = pd.read_csv('../data/exports/complete-country-list.csv')
df_immunization = pd.read_csv('../data/exports/df_finalMergeClean.csv')

### Analysis

In [25]:
# Duplicate dataframe for analysis
df_custCompleteCountryList = df_completeCountryList

# Apply merCol function to check if ISO_A3, if not use GU_A3
df_custCompleteCountryList['ISOA3_GUA3'] = np.vectorize(mergeCol)(df_completeCountryList['ISO_A3'],df_completeCountryList['ADM0_DIF'],df_completeCountryList['LEVEL'],df_completeCountryList['ADM0_A3'],df_completeCountryList['ADM0_A3_IS'],df_completeCountryList['GU_A3'])
df_custCompleteCountryList = df_custCompleteCountryList.drop('Unnamed: 0', axis=1)
df_custCompleteCountryList = df_custCompleteCountryList.rename(columns = {'NAME' : 'name'})

In [26]:
# Export custom complete country list
df_custCompleteCountryList.to_csv('../data/exports/custom-complete-country-list.csv')

In [27]:
# prep immunization dataframe for merge
df_immunization = df_immunization.rename(columns={'iso3':'ISOA3_GUA3'}).drop('Unnamed: 0', axis=1)

In [28]:
# Merge natural earth and immunization
df_custCountryImmunization = df_custCompleteCountryList.merge(df_immunization,on='ISOA3_GUA3',how='outer')
df_custCountryImmunization
df_immunizationExport = df_custCountryImmunization[np.isfinite(df_custCountryImmunization['DTP1-16'])]
df_immunizationExport['ISOA3_GUA3'].nunique()
df_immunizationExport.to_csv('../data/exports/df_immunizationExport.csv')
# df_immunizationExport[df_immunizationExport['ISOA3_GUA3'] == 'PSE']

In [29]:
df_immunizationExport['name']

1                   Antigua
2                   Barbuda
3               Afghanistan
4                    Angola
6                   Albania
8                   Andorra
9      United Arab Emirates
10                Argentina
11                  Armenia
27                Australia
28                  Austria
29               Azerbaijan
30                 Brussels
31                  Flemish
32                  Walloon
33                  Burundi
34                    Benin
35             Burkina Faso
36               Bangladesh
37                 Bulgaria
38           Brcko District
39     Fed. of Bos. & Herz.
40              Rep. Srpska
41                  Bahrain
42                  Bahamas
44                  Belarus
45                   Belize
47                  Bolivia
49                   Brazil
50                 Barbados
               ...         
257              Seychelles
258              UNDOF Zone
259                   Syria
261                    Chad
262                 

In [30]:
df_graphicNoData = df_graphicExample.drop(['amt', 'amt_refugee'], axis = 1)

In [31]:
list(df_graphicNoData)

['name', 'name_abbrev', 'id', 'lat', 'lon']

In [38]:
df_graphicData = df_graphicNoData.merge(df_immunizationExport, on='name', how = 'outer')
df_graphicData.to_csv('../data/exports/final_export.csv')